In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display
import s3fs

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", 
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))
s3.ls('escience2022/Remy/')

['escience2022/Remy/.keep',
 'escience2022/Remy/.wget-20221102193645.sh.status',
 'escience2022/Remy/.wget-20221102193729.sh.status',
 'escience2022/Remy/.wget-20221102193735.sh.status',
 'escience2022/Remy/.wget-20221102193742.sh.status',
 'escience2022/Remy/Era5',
 'escience2022/Remy/dl_era5_hus.py',
 'escience2022/Remy/dl_era5_wind.py',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20150101-20201231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20210101-20301231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20410101-20501231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20510101-20601231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20610101-20701231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20710101-20801231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
 'escience

In [3]:
years = np.arange(2033,2100,1).astype(int)
ssp = 'ssp370'

In [4]:
huspath = list([
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20150101-20201231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20210101-20301231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20410101-20501231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20510101-20601231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20610101-20701231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20710101-20801231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20910101-21001231.nc'
])

vapath = list([
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20150101-20201231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20210101-20301231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20410101-20501231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20510101-20601231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20610101-20701231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20710101-20801231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
 'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20910101-21001231.nc'
])

sopenlisth=[s3.open(ss) for ss in huspath]
sopenlistv=[s3.open(ss) for ss in vapath]

hus370 = xr.open_mfdataset(sopenlisth, chunks={'time':-1,'lat':20,'lon':20})
va370 = xr.open_mfdataset(sopenlistv, chunks={'time':-1,'lat':20,'lon':20})


In [5]:
def compute_ivt1(ivx):
    g = 9.81
    iv_ = -1/g*ivx.qv.integrate(coord='plev')
    return iv_

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        # for each latitude get limit longitudes of shapes
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1: # if there is a shape at that longitude
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10: # avoid filling where only 1 point in shape
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc

In [6]:
# pre-computed 98th percentile IVT
q98 = xr.open_dataset('q93_2000.nc')
q98 = q98.rename({'__xarray_dataarray_variable__':'ivt'})

In [ ]:
for year in years:
    print(year)
    husloc = hus370.sel(time=slice(str(year)+"-01-01", str(year)+"-12-31"))
    valoc = va370.sel(time=slice(str(year)+"-01-01", str(year)+"-12-31"))
    vas = valoc.va
    hus = husloc.hus
    plev = husloc.plev
    lat_ = husloc.lat
    lon_ = husloc.lon
    ivt = compute_ivt(hus,vas,plev)
    husloc.close()
    valoc.close()
    ivt_ns = ivt.copy()
    ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
    ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward
    excess = ivt_ns_pos-q98
    ivt_ns_pos.close()
    ar_points = xr.where(excess>0,1,0)
    out_ar = ar_points.copy()
    ar_points.close()
    #out_ar = out_ar.drop_vars(['quantile','member_id'])
    out_loc = np.zeros((out_ar.ivt.shape[0],out_ar.ivt.shape[1],out_ar.ivt.shape[2])).astype(int)
    test_val = out_ar.ivt.values[:]
    for tt in range(len(out_ar.time)):
        df_loc = test_val[:,tt,:]
        ll = plt.contour(df_loc,levels=[0,1])
        plt.close()
        for item in ll.collections:
            for i in item.get_paths():
                v = i.vertices
                crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
                if (crit>=20): # AR has to be at least 20 deg lat 
                    xx=(v[:, 0]).astype(int)
                    yy=(v[:, 1]).astype(int)
                    for (x,y) in zip(xx,yy):
                        out_loc[y,tt,x] = 1
                        
    out_ar.ivt.values = out_loc.astype(bool)
    out_ar.to_netcdf(str(year)+'_crit_p93_'+ssp+'.nc')
    out_ar.close()
    AR = xr.open_dataset(str(year)+'_crit_p93_'+ssp+'.nc')
    ivt_ = AR.ivt
    ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon')# add extra points for AR at the edge of the world
    out_ar = ivt.copy()
    out_ar.values[:] = out_ar.values[:]*0.0

    for k in range(ivt.shape[0]):
        matrix = ivt[k,:,:]
        tst = floodfill(matrix)
        out_ar[k,:,:] = tst
    
    # add the values of the extra points to get ARs at the end of the world
    out_ar[:,:,:5] = out_ar[:,:,:5]+out_ar[:,:,144:]
    out_ar = out_ar[:,:,:144]
    out_ar = xr.where(out_ar>1,1,out_ar)
    out_ar = out_ar.astype(bool)
    lat_ = AR.lat
    lon_ = AR.lon
    out_ar.to_netcdf(str(year)+'_AR_detection_p93_'+ssp+'.nc')

2033


/tmp/ipykernel_4094/3831064186.py:26: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])


2034
